Подключение необходимых библиотек:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import ndcg_score, average_precision_score

### Шаг 1: Подготовка и проверка датасета



1. **Загрузка данных**: Начнем с загрузки данных из файла intern_task.csv и ознакомления с его структурой и содержимым.


In [ ]:
df = pd.read_csv('intern_task.csv')

# Посмотрим первые 5 строк данных
df.head()

2. **Изучение данных**: Проанализируем основные статистики данных, проверим наличие пропущенных значений и оценим распределение значений признаков.

**df.describe()**: Этот метод выводит основные статистические характеристики числовых признаков в DataFrame, такие как среднее, стандартное отклонение, минимум, максимум и квартили. Это предоставляет общее представление о распределении числовых данных.

In [ ]:
print(df.describe())

**df.nunique()**: Этот метод возвращает количество уникальных значений для каждого признака в DataFrame. Это полезно для определения количества различных категорий или уровней в категориальных признаках.

In [ ]:
print(df.nunique())

**df.isnull().sum()**: Подсчет количества пропущенных значений по каждому признаку.

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cmap='viridis', cbar=False)
plt.title('Пропущенные значения')
plt.show()

Видим, что пропущенных значений не обнаружено, поэтому их обработка не требуется.

Оценка распределения значений признаков:

In [ ]:
# Визуализация распределения числовых признаков
df.hist(bins=20, figsize=(12, 10))
plt.suptitle('Распределение числовых признаков', x=0.5, y=1.02, ha='center', fontsize='large')
plt.tight_layout()
plt.show()

# Визуализация распределения категориальных признаков
for column in df.select_dtypes(include=['object']):
    plt.figure(figsize=(15, 10))
    df[column].value_counts().plot(kind='bar', color='skyblue')
    plt.title(f'Распределение значений признака {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')
    plt.xticks(rotation=45)
    plt.show()


3. **Подготовка данных**: Разделим данные на признаки (X) и целевую переменную (y), а затем отмасштабируем признаки.

In [ ]:
X = df.drop(columns=['rank', 'query_id'])
y = df['rank']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**df.corr()**: Этот метод вычисляет корреляционную матрицу между числовыми признаками в DataFrame. Корреляция показывает степень линейной зависимости между двумя переменными. Значения корреляции находятся в диапазоне от -1 до 1, где близкие к 1 значения указывают на положительную линейную зависимость, значения близкие к -1 указывают на отрицательную линейную зависимость, а значения близкие к 0 указывают на отсутствие линейной зависимости.

In [ ]:
print(df.corr())

Визуализация распределения целевой переменной (rank)

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(y, kde=True)
plt.title('Распределение целевой переменной (rank)')
plt.xlabel('Rank')
plt.ylabel('Частота')
plt.show()


### Шаг 2: Обучение модели ранжирования


1. **Разделение данных**: Разделим данные на обучающий и тестовый наборы.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

2. **Обучение модели**: Обучим выбранную модель на обучающем наборе данных.

In [ ]:
# Создание и обучение модели RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)


### Шаг 3: Оценка модели


1. Получим предсказания рангов на тестовом наборе.

In [ ]:
y_pred = rf_model.predict(X_test)

2. **Вычисление метрики ранжирования NDCG (Normalized Discounted Cumulative Gain)**

NDCG оценивает качество ранжирования путем учета как порядка предсказанных результатов, так и их уровня релевантности.

In [ ]:
ndcg_5 = ndcg_score(np.expand_dims(y_test, axis=0), np.expand_dims(y_pred, axis=0), k=5)
print("NDCG@5 на тестовом наборе данных:", ndcg_5)

3. **Вычисление средней точности Mean Average Precision (MAP)**

MAP оценивает качество ранжирования путем вычисления среднего значения точности (Precision) для каждого уровня релевантности истинных меток (ground truth) на заданном числе верхних позиций (rank).

In [ ]:
y_test_reshape = np.array(y_test).reshape(-1, 1)
y_pred_reshape = np.array(y_pred).reshape(-1, 1)

map_score = average_precision_score(y_test_reshape, y_pred_reshape)
print(f"Mean Average Precision (MAP): {map_score}")

4. **Вычисляем Mean Reciprocal Rank (MRR)**

MRR оценивает качество ранжирования по первому верному результату. Он вычисляется как средний обратный ранг первого верного результата по всем запросам.

In [ ]:
def mean_reciprocal_rank(y_true, y_pred):
    # Сортируем индексы предсказанных значений по убыванию вероятности
    sorted_indexes = np.argsort(y_pred)[::-1]
    
    for idx in sorted_indexes:
        # Находим первый индекс, для которого y_true равно 1 (соответствует релевантному результату)
        if y_true[idx] == 1:
            # Возвращаем обратную величину ранга
            return 1 / (idx + 1)
    
    # Если ни один релевантный результат не найден
    return 0

# Вычисляем взаимные ранги
reciprocal_ranks = [mean_reciprocal_rank(y_test, y_pred) for y_test, y_pred in zip(y_test_reshape, y_pred_reshape)]

# Фильтруем значения None (если они есть)
valid_reciprocal_ranks = [rr for rr in reciprocal_ranks if rr is not None]

# Вычисляем среднее значение действительных взаимных рангов
mrr_score = np.mean(valid_reciprocal_ranks)
print(f"Mean Reciprocal Rank (MRR): {mrr_score}")


### Шаг 4: Сводка результатов

После выполнения всех шагов анализа и обучения модели ранжирования, получены следующие результаты:

1. **Подготовка и проверка датасета**: В процессе подготовки данных были загружены, изучены и предобработаны данные из файла `intern_task.csv`. Датасет содержит числовые и категориальные признаки. Пропущенных значений не обнаружено. Распределение признаков визуализировано для оценки характеристик данных.

2. **Обучение модели ранжирования**: Данные были разделены на обучающий и тестовый наборы. Затем была обучена модель `RandomForestRegressor` на отмасштабированных признаках.

3. **Оценка модели**:
    - **NDCG@5 (Normalized Discounted Cumulative Gain)**: Получено значение NDCG@5 на тестовом наборе данных.
    - **MAP (Mean Average Precision)**: Рассчитано среднее значение точности модели.
    - **MRR (Mean Reciprocal Rank)**: Вычислено среднее взаимное значение рангов первого верного результата.